# all text encoders

In [22]:
import pprint
pp = pprint.PrettyPrinter(indent=4, width=200, depth=None, stream=None, compact=False, sort_dicts=False)

In [23]:
import json

with open('../pdf_to_dictionary/english_syn_ant.json', encoding='utf-8') as f:
    d = json.load(f)

In [24]:
import numpy as np
from numpy.linalg import norm

def my_cosine_similarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

In [25]:
# make smaller test dict
d_keys_li=list(d.keys())
smaller_dict={}
for i in range(0,10):
    smaller_dict[d_keys_li[i]]=d[d_keys_li[i]]
# pp.pprint(smaller_dict)

In [26]:
# d - full test disctionary
# smaller_dict - 10 key dictionary
DICT = smaller_dict
#DICT = d

In [27]:
for baseword,v in DICT.items():
    print(baseword,v["syn"],v["ant"])
    break

aback ['backwards', 'rearwards', 'aft', 'abaft', 'astern', 'behind', 'back'] ['onwards', 'forwards', 'ahead', 'before', 'afront', 'beyond', 'afore']


In [28]:
def get_avg_res_bert(tokenizer, model):
    from scipy.spatial import distance
    import numpy as np
    sum_syn=0
    sum_ant=0
    cnt_syn=0
    cnt_ant=0

    sum=0
    cnt=0
    avg=0

    for baseword,v in DICT.items():
        encoded_input_base=tokenizer(baseword,return_tensors='pt')
        output_base = model(**encoded_input_base)
        embedded_base_avg=np.average(output_base.last_hidden_state[0].detach().numpy(), axis=0)

        for synonym in v["syn"]:
            encoded_input_synonym=tokenizer(synonym,return_tensors='pt')
            output_synonym = model(**encoded_input_synonym)
            embedded_synonym_avg=np.average(output_synonym.last_hidden_state[0].detach().numpy(), axis=0)
            sim_syn=my_cosine_similarity(embedded_base_avg,embedded_synonym_avg)
            sum_syn=sum_syn+sim_syn
            cnt_syn=cnt_syn+1
        
        for antonym in v["ant"]:
            encoded_input_antonym=tokenizer(antonym,return_tensors='pt')
            output_antonym = model(**encoded_input_antonym)
            embedded_antonym_avg=np.average(output_antonym.last_hidden_state[0].detach().numpy(), axis=0)
            sim_ant=my_cosine_similarity(embedded_base_avg,embedded_antonym_avg)
            sum_ant=sum_ant+sim_ant
            cnt_ant=cnt_ant+1
            
        print(cnt_syn, cnt_ant, end="\r")
           
    print("syn", sum_syn/cnt_syn)
    print("ant", sum_ant/cnt_ant)

In [29]:
def get_avg_res_other(tokenizer, model):
    from scipy.spatial import distance
    import numpy as np
    sum_syn=0
    sum_ant=0
    cnt_syn=0
    cnt_ant=0

    sum=0
    cnt=0
    avg=0

    for baseword,v in DICT.items():
        encoded_input_base=tokenizer(baseword,return_tensors='pt')
        output_base = model(**encoded_input_base,output_hidden_states=True)
        output_base_np=output_base.hidden_states[-1][0].detach().numpy()
        embedded_base_avg=np.average(output_base_np, axis=0)

        for synonym in v["syn"]:
            encoded_input_synonym=tokenizer(synonym,return_tensors='pt')
            output_synonym = model(**encoded_input_synonym,output_hidden_states=True)
            output_synonym_np=output_synonym.hidden_states[-1][0].detach().numpy()
            embedded_synonym_avg=np.average(output_synonym_np, axis=0)
            sim_syn=my_cosine_similarity(embedded_base_avg,embedded_synonym_avg)
            sum_syn=sum_syn+sim_syn
            cnt_syn=cnt_syn+1
        
        for antonym in v["ant"]:
            encoded_input_antonym=tokenizer(antonym,return_tensors='pt')
            output_antonym = model(**encoded_input_antonym,output_hidden_states=True)
            output_antonym_np=output_antonym.hidden_states[-1][0].detach().numpy()
            embedded_antonym_avg=np.average(output_antonym_np, axis=0)
            sim_ant=my_cosine_similarity(embedded_base_avg,embedded_antonym_avg)
            sum_ant=sum_ant+sim_ant
            cnt_ant=cnt_ant+1
            
        print(cnt_syn, cnt_ant, end="\r")
           
    print("syn", sum_syn/cnt_syn)
    print("ant", sum_ant/cnt_ant)

In [30]:
# bert-base-multilingual-uncased
# Run time: 21m
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
model = BertModel.from_pretrained("bert-base-multilingual-uncased")

get_avg_res_bert(tokenizer,model)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


syn 0.7700543712585344
ant 0.7717551394043681


In [31]:
# bert-base-multilingual-cased
# Run time: 19m
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")

get_avg_res_bert(tokenizer,model)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


syn 0.457380204709298
ant 0.44675041971919693


In [32]:
# hubert-base-cc
# Run time: 18m
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("SZTAKI-HLT/hubert-base-cc")
tokenizer = AutoTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")

get_avg_res_bert(tokenizer,model)

Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


syn 0.9483029355696582
ant 0.9476099771873974


In [33]:
# text-generation-news-gpt2-small-hungarian
# Run time: 21m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NYTK/text-generation-news-gpt2-small-hungarian")
model = AutoModelForCausalLM.from_pretrained("NYTK/text-generation-news-gpt2-small-hungarian")

get_avg_res_other(tokenizer,model)

syn 0.6119087130651562
ant 0.6028744581146775


In [34]:
# text-generation-poem-petofi-gpt2-small-hungarian
# Run time: 21m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("NYTK/text-generation-poem-petofi-gpt2-small-hungarian")
model = AutoModelForCausalLM.from_pretrained("NYTK/text-generation-poem-petofi-gpt2-small-hungarian")

get_avg_res_other(tokenizer,model)

syn 0.6081417402543059
ant 0.5796928997630271


In [35]:
# mGPT
# Run time: 234m
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/mGPT")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/mGPT")

get_avg_res_other(tokenizer,model)

syn 0.8672215375331563
ant 0.8712536692619324


In [36]:
# sentiment-hts2-xlm-roberta-hungarian
# Run time: 11m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts2-xlm-roberta-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts2-xlm-roberta-hungarian")

get_avg_res_other(tokenizer,model)

syn 0.3226999066291599
ant 0.29995385095233396


In [37]:
# sentiment-hts5-xlm-roberta-hungarian
# Run time: 18m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts5-xlm-roberta-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts5-xlm-roberta-hungarian")

get_avg_res_other(tokenizer,model)

syn 0.756863523513899
ant 0.7665652340260621


In [38]:
# sentiment-hts5-hubert-hungarian
# Run time: 18m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts5-hubert-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts5-hubert-hungarian")

get_avg_res_other(tokenizer,model)

syn 0.8858766801860354
ant 0.8932403487579845


In [39]:
# sentiment-hts2-hubert-hungarian
# Run time: 17m
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NYTK/sentiment-hts2-hubert-hungarian")
model = AutoModelForSequenceClassification.from_pretrained("NYTK/sentiment-hts2-hubert-hungarian")

get_avg_res_other(tokenizer,model)

syn 0.7766083659382041
ant 0.7802397848289704


In [40]:
# xlm-roberta-base
# Run time: 43m
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

get_avg_res_other(tokenizer,model)

syn 0.9947209669909346
ant 0.9946233507628753


In [41]:
# xlm-roberta-large
# Run time: 82m
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

get_avg_res_other(tokenizer,model)

syn 0.9950053538751165
ant 0.9955205042785573


In [42]:
# xlm-mlm-100-1280
# Run time: 109m
from transformers import XLMTokenizer, XLMWithLMHeadModel
import torch

tokenizer = XLMTokenizer.from_pretrained("xlm-mlm-100-1280")
model = XLMWithLMHeadModel.from_pretrained("xlm-mlm-100-1280")

get_avg_res_other(tokenizer,model)

Some weights of XLMWithLMHeadModel were not initialized from the model checkpoint at xlm-mlm-100-1280 and are newly initialized: ['transformer.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


syn 0.5611122723566283
ant 0.5727460027855134


In [43]:
# xlm-roberta-xl
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("facebook/xlm-roberta-xl")
model = AutoModelForMaskedLM.from_pretrained("facebook/xlm-roberta-xl")

get_avg_res_other(tokenizer,model)

syn 0.9468308495819022
ant 0.947439991425131
